In [1]:
%matplotlib inline 
# ensures that plost are embedded in the notebookimport os
import numpy
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab # mlab.normpdf(x, mu, sigma)
import sklearn.datasets
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

<h2>Load data</h2>

In [2]:
cancer = load_breast_cancer()
print('Dataset description:\n\t{0:s}'.format(cancer.DESCR))
print('Number of features: {0:d}'.format(len(cancer.feature_names)))
print('Number of observations {0:d}'.format(cancer.data.shape[0]))
print('Classes:')
for class_name in cancer.target_names:
    print('\t' + class_name)
print('Tumor Features:')
for feature_name in cancer.feature_names:
    print('\t' + feature_name)

Dataset description:
	.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance

<h2>Prepare the data</h2>
<ul>
    <li>Split the data into training and test sets</li>
    <li>Normalize the data using <i>sklearn.preprocessing.StandardScaler:</i><br>
        Each feature is rescaled to have mean of 0 and std of 1    
    </li>
</ul>    

ANN converges much better if the input data is scaled <br>
Make sure to transform training & test data in exactly the same way

In [3]:
X_train, X_test, y_train, y_test = train_test_split(cancer['data'], cancer['target'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)

X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [4]:
X_test.shape

(143, 30)

<h2>Train Multi-Layer Perceptron Classifier</h2>

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
mlp = Sequential()
mlp.add(Dense(15, input_dim=30, activation='relu'))
mlp.add(Dense(10, input_dim=15, activation='relu'))
mlp.add(Dense(1, input_dim=10, activation='sigmoid'))
mlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                465       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 636
Trainable params: 636
Non-trainable params: 0
_________________________________________________________________


In [6]:
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
training_history = mlp.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), verbose=0)

<h2>Test the model</h2>
Reminder: <br>
<b>precision</b>: fraction of the instancies classfied as $Class_x$ that are actually $Class_x$<br>
<b>recall</b>: fraction of the $Class_x$ instancies actually classified as $Class_x$<br>
$F_1 = 2*\frac{precision \times recall}{precision + recall}$

In [8]:
loss, accuracy = mlp.evaluate(X_test,y_test)
print(f'Model score: {accuracy:3.3f} (loss: {loss:3.3f})')
print('---------------------------------------------------------')
from sklearn.metrics import confusion_matrix
prediction = mlp.predict(X_test)>0.5
C = confusion_matrix(y_test,prediction) 
# Cij : number of observations known to be in group i but predicted to be in group j
print('Known/Predicted\t\t{0:s}\t{1:s}'.format(cancer.target_names[0],cancer.target_names[1]))
print('{0:s}\t\t{1:d}\t\t{2:d}'.format(cancer.target_names[0],C[0,0],C[0,1]))
print('{0:s}\t\t\t{1:d}\t\t{2:d}'.format(cancer.target_names[1],C[1,0],C[1,1]))
print('---------------------------------------------------------')
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction, target_names=cancer.target_names,digits=3)) 


5/5 [==============================] - 0s 1ms/step - loss: 0.0882 - accuracy: 0.9650
Model score: 0.965 (loss: 0.088)
---------------------------------------------------------
5/5 [==============================] - 0s 998us/step
Known/Predicted		malignant	benign
malignant		56		3
benign			2		82
---------------------------------------------------------
              precision    recall  f1-score   support

   malignant      0.966     0.949     0.957        59
      benign      0.965     0.976     0.970        84

    accuracy                          0.965       143
   macro avg      0.965     0.963     0.964       143
weighted avg      0.965     0.965     0.965       143



In [9]:
print('Model Weight Dimenensions:')
for i, layer in enumerate(mlp.layers):
    print('\tLayer:{0:d}'.format(i))
    print(f'\t\tWeights shape:{layer.weights[0].shape}')
    print(f'\t\tBiases shape:{layer.weights[1].shape}')

Model Weight Dimenensions:
	Layer:0
		Weights shape:(30, 15)
		Biases shape:(15,)
	Layer:1
		Weights shape:(15, 10)
		Biases shape:(10,)
	Layer:2
		Weights shape:(10, 1)
		Biases shape:(1,)
